In [1]:
from scipy.integrate import solve_ivp
from scipy.linalg import polar
import matplotlib
from matplotlib import pyplot as plt
#use tex font
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams.update({'font.size': 12})
import numpy as np
import vpython as v
#constants
g=1
l=1
m=1
##########

#q =
#    cba11
#    cba21
#    cba31
#    cba12
#    cba22
#    cba32
#    cba13
#    cba23
#    cba33

#helper variables
gvec = np.array([0, -g, 0]).astype(np.complex128)
rpwFb = np.array([l, 0, 0]).astype(np.complex128)
qTest = np.array([11,21,31,12,22,32,13,23,33]).astype(np.complex128)
qdim = qTest.shape[0]
Ze_1x3 = np.zeros([1,3]).astype(np.complex128)
Ze_3x1 = np.zeros([3,1]).astype(np.complex128)
I_3x3 = np.identity(3).astype(np.complex128)
I_1 = I_3x3[0,None].T
I_2 = I_3x3[1,None].T
I_3 = I_3x3[2,None].T

def CrossOp(v):
    skv = np.roll(np.roll(np.diag(v.flatten()), 1, 1), -1, 0)
    return skv - skv.T
#print(CrossOp(v.T))

def evalS(q):
    CbaT = q.reshape([3,3]).T
    cba1 = CbaT[:, 0,None]
    cba2 = CbaT[:, 1,None]
    cba3 = CbaT[:, 2,None]
    S = np.block([
        [Ze_1x3, cba3.T, Ze_1x3],
        [Ze_1x3, Ze_1x3, cba1.T],
        [cba2.T, Ze_1x3, Ze_1x3]
    ])
    #print(S)
    return S

def evalXi(q):
    CbaT = q.reshape([3,3]).T
    cba1 = CbaT[:, 0,None]
    cba2 = CbaT[:, 1,None]
    cba3 = CbaT[:, 2,None]
    Xi = np.block([
        [2 * cba1.T, Ze_1x3, Ze_1x3],
        [Ze_1x3, 2 * cba2.T, Ze_1x3],
        [cba2.T,     cba1.T, Ze_1x3],
        [CrossOp(-1*cba2), CrossOp(cba1), -1*I_3x3]
    ])
    #print(Xi)
    return Xi

#undifferentiated DCM constraint used for stabilizing
def evalPhi(q):
    CbaT = q.reshape([3,3]).T
    cba1 = CbaT[:, 0,None]
    cba2 = CbaT[:, 1,None]
    cba3 = CbaT[:, 2,None]
    Phi = np.block([
        [cba1.T.dot(cba1) - 1],
        [cba2.T.dot(cba2) - 1],
        [cba2.T.dot(cba1)],
        [CrossOp(cba1)@cba2-cba3]
    ])
    return Phi.T[0]

def evalSTMS(q):
    S = evalS(q)
    M = m*l*l*np.identity(3)#*-m*l*l*CrossOp(I_1)@CrossOp(I_1))
    M[0,0] = 0.001*m #-> inertia about an axis of zero seemed to cause the matrix to be singular. DCM wont play nice with 0 inertia along an axis
    STMS = S.T @ M @ S
    return STMS
#print('CROSSY',CrossOp(I_1)@CrossOp(I_1))
#print(qTest.reshape([3,3]).T)
#print(qTest.reshape([3,3]).T@rpwFb)
#print(qTest.reshape([3,3]).T[0,1])

def evalLagrangian(q,qd):
    CbaT = q.reshape([3,3]).T
    Upw = -m*gvec.dot(CbaT @ rpwFb) #where is the sign error coming from? -> i forgot -m*l*l*CrossOp(I_1)@CrossOp(I_1)) is positive along diagonal, but -m*l*l*np.identity(3) isnt
    #print(np.real(Upw))
    STMS = evalSTMS(q)
    Tpwa = 0.5*(qd @ STMS @ qd)
    Lpwa = Tpwa-Upw
    return Lpwa

def q_from_theta(theta):
    q = np.array([
        np.sin(theta),
        -np.cos(theta),
        0,

        np.cos(theta),
        np.sin(theta),
        0,

        0,
        0,
        1,
    ])
    return q
###############################
STMS_template = evalSTMS(qTest)
Xi_template = evalXi(qTest)
Xidim = Xi_template.shape[0] #number of lagrange multipliers
I_qdim = np.identity(qdim)
Ze_Xidim = np.zeros([Xidim,Xidim])
###############################
#numerical differencing
hstep = 0.00000000001
def evalSTMS_dot(q,qd):
    #use complex step to get the time derivative. based on multivariate chain rule
    M_dot = np.zeros_like(STMS_template,dtype=np.complex128)
    for col in range(qdim):
        M_dot += (np.imag(evalSTMS(q+I_qdim[col]*hstep*1j))/hstep)*qd[col]
    return M_dot
def evalXi_dot(q,qd):
    #use complex step to get the time derivative. based on multivariate chain rule
    Xi_dot = np.zeros_like(Xi_template,dtype=np.complex128)
    for col in range(qdim):
        Xi_dot += (np.imag(evalXi(q+I_qdim[col]*hstep*1j))/hstep)*qd[col]
    return Xi_dot
def evalLpqT(q,qd):
    LpqT = np.zeros(qdim,dtype=np.complex128)
    for col in range(qdim):
        LpqT[col] = np.imag(evalLagrangian(q+I_qdim[col]*hstep*1j,qd))/hstep
    return LpqT

#Q = [q,qd]
def dQdt(t,Q):
    #print(t)
    q = Q[0:qdim]
    CbaT = q.reshape([3,3]).T
    #u, p = polar(CbaT)
    # print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    # print(np.real(q))
    # q = u.T.flatten()
    # print(np.real(q))
    #print(np.real(CbaT))
    #print(np.real(u))
    ##print('DONE')
    #print('projecting')
    #print(np.real(q))
    #q = p.reshape([qdim])
    #print(np.real(q))
    #q = p.reshape(qdim)

    qd = Q[qdim:qdim*2]
    #print(q)
    #evaluate necessary stuff
    Xi= evalXi(q)
    Phi = evalPhi(q)
    STMS= evalSTMS(q)
    STMS_dot = evalSTMS_dot(q,qd)
    Xi_dot = evalXi_dot(q,qd)
    LpqT=evalLpqT(q,qd)
    A = np.block([
        [STMS,-Xi.T],
        [-Xi,Ze_Xidim]
    ])
    #print('DET A',np.linalg.det(np.real(A)))
    #print(np.linalg.det(np.real(Xi@Xi.T)))
    #print(A)
    # print(LpqT.shape)
    # print((STMS_dot@qd).shape)
    # print((Xi_dot@qd).shape)
    alpha = 0.2
    beta = alpha*alpha
    b = np.concatenate((
        LpqT-STMS_dot@qd,
        #Investigation_on_the_Baumgarte_Stabilization_Method_for_Dynamic_Analysis_of_Constrained_Multibody_Systems
        Xi_dot@qd+Xi@qd*alpha*2+Phi*beta
    ))
    # print(A.shape)
    # print(b.shape)
    solN = np.linalg.solve(A,b)
    Qd = np.concatenate((qd,solN[0:qdim]))
    return Qd

#ground truth pendulum solution
def PendulumTheta(t, theta):
    return [
            #d/dt(position)
            theta[1],
            #d/dt(velocity)
            (-g/l)*np.sin(theta[0])
            ]

ODE_tspan = [0, 100]
ODE_teval = np.linspace(ODE_tspan[0],ODE_tspan[1],1000)
ODE_IC = [np.pi*0.5,0]
qd = np.zeros(qdim)
#qd[2] = 1
DCM_IC = np.concatenate((q_from_theta(ODE_IC[0]),qd))
print(DCM_IC)
sol_theta = solve_ivp(
    fun=PendulumTheta,
    t_span=ODE_tspan,
    y0=ODE_IC,
    t_eval = ODE_teval,
    #max_step = 0.05
)
#it looks like this actually does work.
sol_DCM = solve_ivp(
    fun=dQdt,
    t_span=ODE_tspan,
    y0=DCM_IC,
    t_eval = ODE_teval,
    #max_step = 0.05
)
print('num fev:',sol_DCM.nfev)

######################## 3D viz
v.scene.width = v.scene.height = 600

class PendulumSolution:
  def __init__(self,Zoffset,name,color):
    self.name = name

    self.W = v.sphere(pos=v.vec(0,0,Zoffset),
                  radius=0.1,
                  color=v.vec(0.4, 0.4, 0.5))

    self.P = v.sphere(pos=v.vec(l,0,Zoffset),
                  radius=0.1,
                  color=v.vec(color[0],color[1],color[2]),
                  make_trail=True,
                  interval=1,
                  retain=50)
    self.bar = v.cylinder( pos = v.vec(0,0,Zoffset),
                  size=v.vec(1, 0.05, 0.05),
                  color=v.vec(0.4, 0.4, 0.5))
    self.bar.start = self.W
    self.bar.end = self.P
    self.tip_positions = np.zeros([ODE_teval.shape[0],3])
    self.tip_positions[:,2] = Zoffset

GT = PendulumSolution(-0.1,'GT',[1,0,0])
GT.tip_positions[:,0] = np.sin(sol_theta.y[0])*l
GT.tip_positions[:,1] = -np.cos(sol_theta.y[0])*l

DCMpen = PendulumSolution(0.1,'GT',[0,1,0])
print(sol_DCM.y.shape,ODE_teval.shape)
for tindex,t in enumerate(sol_DCM.t):
    q = sol_DCM.y[0:qdim,tindex]
    CbaT = q.reshape([3,3]).T
    pos = CbaT@np.array([l,0,0])
    DCMpen.tip_positions[tindex,:] = pos
#print(DCMpen.tip_positions[:,1])
b1 = v.arrow(
    pos=v.vector(0,0,0),
    axis=v.vector(1,0,0),
    shaftwidth=0.01,
    color=v.color.red)
b2 = v.arrow(
    pos=v.vector(0,0,0),
    axis=v.vector(1,0,0),
    shaftwidth=0.01,
    color=v.color.green)
b3 = v.arrow(
    pos=v.vector(0,0,0),
    axis=v.vector(1,0,0),
    shaftwidth=0.01,
    color=v.color.blue)

solns = [GT,DCMpen]

#####################display

# fig = plt.figure()
# ax = plt.subplot(111)
# for line in range(sol_theta.y.shape[0]):
#     ax.plot(sol_theta.t, sol_theta.y[line], label='$y_{' + str(line) + '}$')
# ax.legend(loc = 'upper right')
# plt.title('Pendulum')
# plt.ylabel('y')
# plt.xlabel('t (s)')
# plt.show()


rate = 60
playback_rate = 3 #x real time
playback_dt = playback_rate/rate
t = 0
time = ODE_tspan[0]

<IPython.core.display.Javascript object>

[ 1.000000e+00 -6.123234e-17  0.000000e+00  6.123234e-17  1.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00]


C:\miniconda\envs\tensorflow_gpuenv\lib\site-packages\numpy\core\numeric.py:501: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


num fev: 1340


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(18, 1000) (1000,)


In [ ]:
while True:
    #print('t')
    v.rate(rate)

    for sol in solns:
        curr_pos_x = np.interp(time, sol_theta.t, sol.tip_positions[:, 0])
        curr_pos_y = np.interp(time, sol_theta.t, sol.tip_positions[:, 1])
        curr_pos_z = np.interp(time, sol_theta.t, sol.tip_positions[:, 2])

        sol.P.pos = v.vec(
            curr_pos_x,
            curr_pos_y,
            curr_pos_z
        )
        sol.bar.axis = sol.bar.end.pos - sol.bar.start.pos

    cba11= np.interp(time, sol_DCM.t, sol_DCM.y[0])
    cba21= np.interp(time, sol_DCM.t, sol_DCM.y[1])
    cba31= np.interp(time, sol_DCM.t, sol_DCM.y[2])
    cba12= np.interp(time, sol_DCM.t, sol_DCM.y[3])
    cba22= np.interp(time, sol_DCM.t, sol_DCM.y[4])
    cba32= np.interp(time, sol_DCM.t, sol_DCM.y[5])
    cba13= np.interp(time, sol_DCM.t, sol_DCM.y[6])
    cba23= np.interp(time, sol_DCM.t, sol_DCM.y[7])
    cba33= np.interp(time, sol_DCM.t, sol_DCM.y[8])

    b1.pos = DCMpen.P.pos
    b2.pos = DCMpen.P.pos
    b3.pos = DCMpen.P.pos

    b1.axis = v.vec(cba11, cba21, cba31)
    b2.axis = v.vec(cba12, cba22, cba32)
    b3.axis = v.vec(cba13, cba23, cba33)

    t = (t + playback_dt)%(ODE_tspan[1]-ODE_tspan[0])
    time = ODE_tspan[0]+t